Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

Data Collection and Processing

In [3]:
# load data
dataset = pd.read_csv('sales_dataset.csv')
dataset.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'sales_dataset.csv'

In [ ]:
# remove duplicate entries
dataset.drop_duplicates(inplace = True)
dataset.shape

(286392, 36)

Data Preprocessing

In [ ]:
# keep relevant features
dataset = dataset[['month', 'category', 'Gender', 'age', 'Region', 'State', 'price', 'Discount_Percent',  'qty_ordered']]
dataset.head(3)

,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total,qty_ordered
0,Oct-2020,Men's Fashion,F,43.0,South,OK,89.9,1798.0,0.0,0.0,1798.0,22.0
1,Oct-2020,Men's Fashion,F,43.0,South,OK,19.0,190.0,0.0,0.0,190.0,12.0
2,Oct-2020,Men's Fashion,F,43.0,South,OK,149.9,1199.2,0.0,0.0,1199.2,10.0


In [ ]:
dataset['State'].value_counts()

NameError: name 'dataset' is not defined

In [ ]:
# seperate month and year, drop years
dataset[['month', 'year']] = dataset.month.str.split("-", expand=True)
dataset.drop('year', axis=1, inplace=True)

In [ ]:
# get information about the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286392 entries, 0 to 286391
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   month             286392 non-null  object 
 1   category          286392 non-null  object 
 2   Gender            286392 non-null  object 
 3   age               286392 non-null  float64
 4   Region            286392 non-null  object 
 5   State             286392 non-null  object 
 6   price             286392 non-null  float64
 7   value             286392 non-null  float64
 8   Discount_Percent  286392 non-null  float64
 9   discount_amount   286392 non-null  float64
 10  total             286392 non-null  float64
 11  qty_ordered       286392 non-null  float64
dtypes: float64(7), object(5)
memory usage: 28.4+ MB


###### Categorical Features - months | category | gender | region | state 

Label Encoding

In [ ]:
encoder = LabelEncoder()

In [ ]:
cat_features = ['month', 'category', 'Gender', 'Region', 'State']

for cat_feature in cat_features:
    dataset[cat_feature] = encoder.fit_transform(dataset[cat_feature])

In [ ]:
dataset.head(3)

,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total,qty_ordered
0,10,8,0,43.0,2,36,89.9,1798.0,0.0,0.0,1798.0,22.0
1,10,8,0,43.0,2,36,19.0,190.0,0.0,0.0,190.0,12.0
2,10,8,0,43.0,2,36,149.9,1199.2,0.0,0.0,1199.2,10.0


Split train and test dataset

In [ ]:
X = dataset.drop(columns='qty_ordered', axis=1)
Y = dataset['qty_ordered']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

Model Training

In [ ]:
regressor = XGBRegressor()

In [ ]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

Evaluation

In [ ]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)
# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)
print('R Squared value = ', r2_train)

R Squared value =  0.9994047945928058


In [ ]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)
print('R Squared value = ', r2_test)
X_test

R Squared value =  0.9861241953554252


,month,category,Gender,age,Region,State,price,value,Discount_Percent,discount_amount,total
156966,3,1,0,69.0,2,2,39.683,396.83,0.0,0.000,396.830
281373,11,9,1,39.0,0,15,10499.900,10499.90,0.0,0.000,10499.900
162848,6,9,0,67.0,3,26,1979.000,1979.00,0.0,0.000,1979.000
199242,7,1,0,29.0,2,20,47.000,47.00,0.0,0.000,47.000
40859,9,8,1,25.0,1,30,120.000,120.00,0.0,0.000,120.000
...,...,...,...,...,...,...,...,...,...,...,...
32197,9,8,0,60.0,2,18,59.900,59.90,0.0,0.000,59.900
105309,2,1,1,25.0,0,22,109.900,109.90,0.0,0.000,109.900
82544,2,8,1,56.0,0,35,29.900,29.90,15.0,4.485,25.415
48021,0,13,0,51.0,2,20,2.600,52.00,0.0,0.000,52.000


In [ ]:
import pickle 
pickle_out = open("regressor.pkl", "wb")
pickle.dump(regressor, pickle_out)


In [ ]:
import numpy as np

test_data = pd.DataFrame([[10,8,0,43.0,2,36,89.9,1798.0,0.0,0.0,1798.0]])
regressor.predict(test_data)

array([20.969732], dtype=float32)